In [1]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
import time

In [2]:
#to keep things in order, and to avoid to copy and paste everytime our functions if we want to use them in more than one folder,
#we can temporarily use this library. 
import sys

#in this way Python will search the implementations also in the path '../HelperFunctions'
sys.path.insert(0, '../HelperFunctions')
sys.path.insert(0, '../pre-processing/Clean_Data/')

In [3]:
from proj1_helpers import *
from common_functions import *
from counters import *
from remove import *
from replace import *
from regressors import batch_iter

In [4]:
yb, input_data, ids = load_csv_data("../data/train.csv", sub_sample=False)

In [5]:
#this will surely be deleted, in this way we are sure that original_data is the original version of the data and we don't have
#to load them again
from copy import deepcopy
originalData = deepcopy(input_data)
originalY = deepcopy(yb)
print(yb)

idx_wrong=np.where(yb==-1)
print(idx_wrong)
yb[idx_wrong]=0
print(yb)

[ 1. -1. -1. ...  1. -1. -1.]
(array([     1,      2,      3, ..., 249996, 249998, 249999]),)
[1. 0. 0. ... 1. 0. 0.]


# Functions from lab 5

In [6]:
def sig(t):
    """apply sigmoid function on t."""
    return np.exp(t)/(1 + np.exp(t))


sigmoid = np.vectorize(sig)

def sigmoid(t):
     return .5*(1+np.tanh(.5 * t))

#Is it okay the multiplication between vectors??
# Why do you divide by the number of samples? (mean)
#def calculate_loss(y, tx, w):
#   """compute the cost by negative log likelihood."""
 #   pred = sigmoid(tx.dot(w))
 #   correctZero = 1e-7
   # loss = np.sum((1 - y) * np.log(1 - pred + correctZero) + y * np.log(pred + correctZero))
    #return np.squeeze(- loss)
    #return -loss/y.shape[0]

# Just to try again the function of the professor --> good idea to add a correction to avoid the log(0)
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    pred = sig(tx.dot(w))
    correctZero = 1e-7
    loss = y.T.dot(np.log(pred+correctZero)) + (1 - y).T.dot(np.log(1 - pred+correctZero))
    return np.squeeze(-loss)

# same question as above
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    pred = sigmoid(tx.dot(w))
    grad = tx.T.dot(pred - y)
    return grad
    #return grad/y.shape[0]

def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    #batch = next(batch_iter(y, tx, 32))
    #minibatch_y, minibatch_tx = batch[0], batch[1]
    grad = calculate_gradient(y, tx, w)
    loss = calculate_loss(y, tx, w)
    w = w - gamma * grad
    return loss, w

In [ ]:
#Creation of tx

input_data = deepcopy(originalData)
print(input_data.shape)
#Clean the dataset
numInvalidValues = countInvalid(input_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(input_data,-999,idxCols)

#standardize
tx,_,_ = standardize(input_data)

#add ones
tx = np.c_[np.ones((yb.shape[0], 1)), tx]
y = yb
print(tx.shape)

In [ ]:
#first tries with gradient descent

#Y, TX = sample_data(y, tx, 10, 150000)
max_iter = 100
w = np.ones(tx.shape[1])
gamma = 10000
loss0 = 0

for iter in range(max_iter):
    w_old = w
    loss, w = learning_by_gradient_descent(y, tx, w, gamma)
    #gamma=0.55*gamma
    if iter%10==0:
        w_new = w
        print(f'Iter = {iter}, Loss = {loss}, |w_old - w_new| = {np.linalg.norm(w_old-w_new)}')




Let's try to generate a submission

In [ ]:
WWW = w 

In [ ]:
xxx, test_data, ids = load_csv_data("../data/test.csv", sub_sample=False)

In [ ]:
#we need to preprocess the test_data as we preprocessed the train data

numInvalidValues = countInvalid(test_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
test_data = replaceWithZero(test_data,-999,idxCols)
txTest,_,_ = standardize(test_data)
txTest = np.c_[np.ones((xxx.shape[0], 1)), txTest]

In [ ]:
y_pred = predict_labels(WWW, txTest)

In [ ]:
create_csv_submission(ids, y_pred, 'submission.csv')

# Same thing with PCA

In [7]:
sys.path.insert(0, '../pre-processing/PCA/')

In [ ]:
from pca_functions import PCAWithCovariance

input_data = deepcopy(originalData)
y = deepcopy(originalY)
print(input_data.shape)
print(y.shape)
numInvalidValues = countInvalid(input_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(input_data,-999,idxCols)
input_data,_,_ = standardize(input_data)

In [9]:
from pca_functions import PCAWithCovariance

input_data = deepcopy(originalData)
y = deepcopy(originalY)
print(input_data.shape)
print(y.shape)
input_data=removeColumns(input_data,0)
print(input_data.shape)
input_data,_,_ = standardize(input_data)

(250000, 30)
(250000,)
(250000, 19)


In [10]:
#Generating the principal components

_,eV = PCAWithCovariance(input_data)

N = 7 #num p. components
components = np.empty(input_data.shape[0])
for i in range(N):
    components = np.c_[components, input_data.dot(eV[:,i])]
    
print(components.shape)
print(components[:,0])
tx = np.c_[np.ones(input_data.shape[0]), components]
print(tx.shape)
print(tx[:,0])
print(tx[:,1])

(250000, 8)
[ 1. -1. -1. ...  1. -1. -1.]
(250000, 9)
[1. 1. 1. ... 1. 1. 1.]
[ 1. -1. -1. ...  1. -1. -1.]


In [ ]:
Y, TX = sample_data(y, tx, 1, 70000)
TX.shape

In [12]:
max_iter = 1000
w = np.ones(tx.shape[1])
gamma = 10000
loss0 = 0
for iter in range(max_iter):
    wold = w
    loss, w = learning_by_gradient_descent(yb, tx, w, gamma)
    #gamma=0.55*gamma
    if iter%10==0:
        wnew = w
        print(f'Iter = {iter}, Loss = {loss}, |wold - wnew| = {np.linalg.norm(wold-wnew)}')
print('final loss =',loss)

Iter = 0, Loss = 290583.38562255097, |wold - wnew| = 1559667685.5938625


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iter = 10, Loss = nan, |wold - wnew| = 11127520.461615779
Iter = 20, Loss = nan, |wold - wnew| = 3336698.4158491185
Iter = 30, Loss = nan, |wold - wnew| = 2057369.6981772936
Iter = 40, Loss = nan, |wold - wnew| = 1152733.7888029253
Iter = 50, Loss = nan, |wold - wnew| = 949252.6802280458
Iter = 60, Loss = nan, |wold - wnew| = 799069.5013994048
Iter = 70, Loss = nan, |wold - wnew| = 654771.4397175333
Iter = 80, Loss = nan, |wold - wnew| = 654771.4397175333
Iter = 90, Loss = nan, |wold - wnew| = 411349.88130415184
Iter = 100, Loss = nan, |wold - wnew| = 411349.88130415184
Iter = 110, Loss = nan, |wold - wnew| = 411349.88130415184
Iter = 120, Loss = nan, |wold - wnew| = 411349.88130415184
Iter = 130, Loss = nan, |wold - wnew| = 217488.530118383
Iter = 140, Loss = nan, |wold - wnew| = 217488.530118383
Iter = 150, Loss = nan, |wold - wnew| = 217488.530118383
Iter = 160, Loss = nan, |wold - wnew| = 217488.530118383
Iter = 170, Loss = nan, |wold - wnew| = 217488.530118383
Iter = 180, Loss = n

KeyboardInterrupt: 

In [ ]:
# generate submission with PCA
xxx, test_data, ids = load_csv_data("../data/test.csv", sub_sample=False)



In [ ]:
numInvalidValues = countInvalid(test_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
test_data = replaceWithZero(test_data,-999,idxCols)
test_data,_,_ = standardize(test_data)

components = np.empty(test_data.shape[0])
for i in range(N):
    components = np.c_[components, test_data.dot(eV[:,i])]
    
#tx_test = np.c_[np.ones(test_data.shape[0]), components]
tx_test = components

y_pred = predict_labels(w, tx_test)
create_csv_submission(ids, y_pred, 'submission.csv')